# Microtuble Analysis Project - Track Feature Extraction

### Imports

In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

# Used for storing tiff movie series and images
import pims

#Used to analyze and extract features from images
from skimage import io
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.util import random_noise

# Used to analyze trajectories
import trackpy as tp
import math

# Used for motion analysis
from scipy.spatial import distance
from numpy import linalg as LA

#Used for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import pickle

from scipy.ndimage.morphology import distance_transform_edt
import math


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def identifyCellFromTrack(track_df, lab):
    cell_ids = np.unique(lab)
    cell_tracks = {}
    for i in range(len(cell_ids)):
        cell_tracks[i] = []
    
    for index,rows in track_df.iterrows():
        try:
            particle_coord_x = int(np.floor(rows['x']))
            particle_coord_y = int(np.floor(rows['y']))
            label_id = lab[particle_coord_y,particle_coord_x]
            cell_tracks[label_id].append(rows['particle'])
        except:
            pass
    
    all_tracks = []    
    tracks_to_remove = []
    
    for i in range(len(cell_ids)):
        cell_tracks[i] = list(set(cell_tracks[i]))
        all_tracks.extend(cell_tracks[i])
        if i==0:
            tracks_to_remove.extend(cell_tracks[i])
    
    dupes = set([x for x in all_tracks if all_tracks.count(x) > 1])
    tracks_to_remove.extend(dupes)
    tracks_to_remove.extend(np.setdiff1d(list(track_df.particle), all_tracks))
    tracks_to_remove = set(tracks_to_remove)
    new_df = track_df[~track_df['particle'].isin(tracks_to_remove)]
    new_df.index = range(len(new_df))
    new_df['cell_id'] = np.nan
    
    reverse_dict = {}
    for key,val in cell_tracks.items():
        for item in val:
            reverse_dict[item] = key

    for index,rows in new_df.iterrows():
        new_df.loc[index, 'cell_id'] = reverse_dict[rows['particle']]
        reverse_dict[rows['particle']]
    return new_df

In [15]:
# Method: Generate feature dataframe that is empty
def generate_features_df(track_df):
    unique_tracks = np.array(track_df['particle'].unique())
    new_df = pd.DataFrame(columns=['ID'])
    for row_num, track_id in enumerate(unique_tracks):
        new_df.loc[row_num] = [track_id]
    new_df = new_df.set_index('ID')
    return new_df

# Add motions features to the features dataframe generated from method above
def calc_motion_features(track_df, traj_feature_df):
    new_df = pd.concat([traj_feature_df, 
                        pd.DataFrame(columns=['speed', 'speed_stdev', 'new_speed', 'new_speed_stdev', 'new_speed_range',
                                              'new_speed_min', 'new_speed_max','curvature', 'curv_stdev', 
                                              'net_displacement', 'new_displacement', 'path_length', 'persistance', 
                                              'new_pathlength', 'new_persistance'])])
    unique_tracks = np.array(track_df['particle'].unique())
    
    for row_num, track_id in enumerate(unique_tracks):
        temp_df = track_df.loc[track_df['particle'] == track_id]

        # Compute Vel X, Vel Y, and Speed
        vel_x = np.gradient(temp_df['x'])
        vel_y = np.gradient(temp_df['y'])
        velocity = np.transpose(np.array([vel_x, vel_y]))
        speed = [np.sqrt(dx**2 + vel_y[i]**2) for i,dx in enumerate(vel_x)]
        
        # Compute Normalized Speed
        new_vel_x = np.gradient(temp_df['new_x'])
        new_vel_y = np.gradient(temp_df['new_y'])
        framerate = np.array(temp_df['delta_t'])
        new_velocity = np.transpose(np.array([new_vel_x, new_vel_y]))
        new_speed = [np.sqrt(dx**2 + new_vel_y[i]**2) for i,dx in enumerate(new_vel_x)]
        new_speed = np.divide(new_speed, framerate)
                              
        # Compute the Acceleration Vector
        #acc_x = np.gradient(vel_x)
        #acc_y = np.gradient(vel_y)
        #acceleration = np.transpose(np.array([acc_x, acc_y]))

        # Compute Unit Norm and Unit Tangent Vectors 
        #unit_tangent = np.multiply(np.transpose(np.repeat([np.divide(1,speed)],2, axis=0)),velocity)

        #norm = np.transpose(np.array([np.gradient(unit_tangent[:,0]), np.gradient(unit_tangent[:,1])]))
        #norm_magnitude = [np.linalg.norm(vector) for vector in norm]
        #unit_norm = np.multiply(np.transpose(np.repeat([np.divide(1,norm_magnitude)],2, axis=0)),norm)

        # Compute Kurvature
        #curv = np.divide(norm_magnitude, speed)
        
        #Compute Curvature again
        ind_to_remove = np.logical_and(np.isnan(temp_df['x']), np.isnan(temp_df['y']))
        temp_x = temp_df['x'][~ind_to_remove]
        temp_y = temp_df['y'][~ind_to_remove]
        t = np.arange(0,len(temp_x))
    
        z_x = np.polyfit(t, temp_x, 3)
        z_y = np.polyfit(t, temp_y, 3)
        z_dx = np.polyder(z_x)
        z_ddx = np.polyder(z_dx)
        z_dy = np.polyder(z_y)
        z_ddy = np.polyder(z_dy)
        
        f_dx = np.poly1d(z_dx)
        f_ddx = np.poly1d(z_ddx)
        f_dy = np.poly1d(z_dy)
        f_ddy = np.poly1d(z_ddy)
        
        t2 = np.linspace(1, len(temp_x)-2, 10)

        dx = f_dx(t2)
        ddx = f_ddx(t2)
        dy = f_dy(t2)
        ddy = f_ddy(t2)
        
        curv = np.divide(np.sqrt((ddy*dx - ddx*dy)**2),np.power((dx**2 + dy**2),1.5))
        #curv = curv[1:-1]

        #dx = np.gradient(temp_df['x'])
        #dy = np.gradient(temp_df['y'])
        #ddx = np.gradient(dx)
        #ddy = np.gradient(dy)
        
        #curv = np.sqrt(np.divide(((ddy*dx - ddx*dy)**2),np.power((dx**2 + dy**2),1.5)))
        
        # Compute Net Displacement
        first_val = temp_df.iloc[0]
        last_val = temp_df.iloc[len(temp_df)-1]
        net_displacement = np.sqrt(np.power(last_val['x'] - first_val['x'],2) + np.power(last_val['y'] - first_val['y'],2))
        new_displacement = np.sqrt(np.power(last_val['new_x'] - first_val['new_x'],2) + np.power(last_val['new_y'] - first_val['new_y'],2))
        
        # Compute Total Path length
        dx_squared = np.power(np.diff(temp_df['x']),2)
        dy_squared = np.power(np.diff(temp_df['y']),2)
        path_length = sum(np.sqrt(dx_squared+dy_squared))

        #Compute New Path Length
        new_dx_squared = np.power(np.diff(temp_df['new_x']),2)
        new_dy_squared = np.power(np.diff(temp_df['new_y']),2)
        new_pathlength = sum(np.sqrt(new_dx_squared+new_dy_squared))

        # Compute Persistence
        persistance = np.divide(net_displacement, path_length)    
        new_persistance = np.divide(new_displacement, new_pathlength)
        # Filter outliers of Acceleration/Speed
        #speed_sorted = sorted(speed)
        #speed_sorted_filtered = speed_sorted[math.floor(len(speed)/5):len(speed)-math.floor(len(speed)/5)]

        #curv_sorted = sorted(curv)
        #curv_sorted_filtered = curv_sorted[math.floor(len(curv)/5):len(curv)-math.floor(len(curv)/5)]

        # Compute Average Acceleration/Speed
        avg_speed = np.average(speed)
        stdev_speed = np.std(speed)
        avg_curv = np.max(curv)
        stdev_curv = np.std(curv)
        new_avg_speed = np.average(new_speed)
        new_stdev_speed = np.std(new_speed)
                              
        # Add values to dataframe
        new_df.loc[track_id]['speed'] = avg_speed
        new_df.loc[track_id]['speed_stdev'] = stdev_speed
        new_df.loc[track_id]['new_speed'] = new_avg_speed
        new_df.loc[track_id]['new_speed_stdev'] = new_stdev_speed
        new_df.loc[track_id]['curvature'] = avg_curv
        new_df.loc[track_id]['curv_stdev'] = stdev_curv
        new_df.loc[track_id]['net_displacement'] = net_displacement
        new_df.loc[track_id]['path_length'] = path_length
        new_df.loc[track_id]['persistance'] = persistance
        new_df.loc[track_id]['new_displacement'] = new_displacement
        new_df.loc[track_id]['new_pathlength'] = new_pathlength
        new_df.loc[track_id]['new_persistance'] = new_persistance
        new_df.loc[track_id]['new_speed_range'] = np.max(new_speed) - np.min(new_speed)
        new_df.loc[track_id]['new_speed_min'] = np.min(new_speed)
        new_df.loc[track_id]['new_speed_max'] = np.max(new_speed)
    return new_df

# Add the particle mass feature to the features dataframe 
def calc_mass_features(track_df, traj_feature_df):
# Parse CSV file to find amplitude average + stdev
    unique_tracks = np.array(track_df['particle'].unique())
    new_df = pd.concat([traj_feature_df, 
                        pd.DataFrame(columns=['avg_mass', 'stdev_mass'])])

    for i, track_id in enumerate(unique_tracks):
        temp_df = track_df.loc[track_df['particle'] == track_id]

        mass = np.array(temp_df['mass'])
        mass_sorted = sorted(mass)
        mass_sorted_filtered = mass_sorted[math.floor(len(mass)/5):len(mass)-math.floor(len(mass)/5)]

        mass_avg = np.average(mass_sorted_filtered)
        mass_stdev = np.std(mass_sorted_filtered)

        new_df.loc[track_id]['avg_mass'] = mass_avg
        new_df.loc[track_id]['stdev_mass'] = mass_stdev
    return new_df

In [16]:
#Function Filters the track matrix (ie. t1,t2) using features
def filter_track_from_feat(track_df, feature_df, feature_name, min_val=-np.inf, max_val=np.inf):
    tracks_to_remove = []
    for index,vals in feature_df.iterrows():
        if (vals[feature_name] < min_val or vals[feature_name]>max_val):
            tracks_to_remove.append(index)
    #print ("deleting following tracks:" , tracks_to_remove)
    new_df = track_df[~track_df['particle'].isin(tracks_to_remove)]
    return new_df    
    
#Function filteres the trajectory feature matrix using features
def filter_traj_from_feat(feature_df, feature_name, min_val=-np.inf, max_val=np.inf):
    tracks_to_remove = []
    for index,vals in feature_df.iterrows():
        if (vals[feature_name] < min_val or vals[feature_name]>max_val):
            tracks_to_remove.append(index)
    #print ("deleting following tracks:" , tracks_to_remove)
    new_df = feature_df[~feature_df.index.isin(tracks_to_remove)]
    return new_df

In [17]:
def dist(x1,y1, x2,y2, x3,y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1
    something = px*px + py*py
    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(something)
    if u > 1:
        u = 1
    elif u < 0:
        u = 0
    x = x1 + u * px
    y = y1 + u * py
    dx = x - x3
    dy = y - y3
    dist = math.sqrt(dx*dx + dy*dy)
    return dist


In [18]:
# Distance Functions
# Gets the distance between the axis and a specific track
def getDistanceFromAxis(track_df, track_id,  cell_ID=1, axis='major', label_img=None,):
    props = regionprops(label_img)[cell_ID-1]
    y1, x1 = props.centroid
    orientation = props.orientation
    x2 = np.nan
    y2 = np.nan
    if axis=='major':
        x2 = x1 + math.cos(orientation) * 0.5 * props.major_axis_length
        y2 = y1 - math.sin(orientation) * 0.5 * props.major_axis_length
    else:
        x2 = x1 - math.sin(orientation) * 0.5 * props.minor_axis_length
        y2 = y1 - math.cos(orientation) * 0.5 * props.minor_axis_length
        
    temp_df = track_df.loc[track_df['particle'] == track_id]
    distances = []
    for index,rows in temp_df.iterrows():
        x0 = rows['x']
        y0 = rows['y']

        num = np.absolute(np.multiply(y2-y1, x0) - np.multiply(x2-x1, y0)+ np.multiply(x2,y1) - np.multiply(y2,x1))
        den = np.sqrt(np.power(y2-y1,2) + np.power(x2-x1,2))
        distance = np.divide(num,den)
        distances.append(distance)
        
    return min(distances)


#Gets the signed distance between an axis and a specific cell (MIGHT NOT NEED THIS)
def getsignedDistanceFromAxis(track_df, track_id, label_img=None, cell_ID=1, axis='major'):
    props = regionprops(label_img)[cell_ID-1]
    y1, x1 = props.centroid
    orientation = props.orientation
    x2 = np.nan
    y2 = np.nan
    if axis=='major':
        x2 = x1 + math.cos(orientation) * 0.5 * props.major_axis_length
        y2 = y1 - math.sin(orientation) * 0.5 * props.major_axis_length
    else:
        x2 = x1 - math.sin(orientation) * 0.5 * props.minor_axis_length
        y2 = y1 - math.cos(orientation) * 0.5 * props.minor_axis_length
        
    temp_df = track_df.loc[track_df['particle'] == track_id]
    distances = []
    for index,rows in temp_df.iterrows():
        x0 = rows['x']
        y0 = rows['y']
        #print(index,x0,y0)

        num = np.multiply(y2-y1, x0) - np.multiply(x2-x1, y0)+ np.multiply(x2,y1) - np.multiply(y2,x1)
        den = np.sqrt(np.power(y2-y1,2) + np.power(x2-x1,2))
        distance = np.divide(num,den)
        distances.append(distance)
        
    return min(distances)

# Compute Distance of a track from the center of the cell
def getDistanceFromCenter(track_df, track_id, cell_ID=1, label_img=None):
    distances = [] 
    y0, x0 = regionprops(img_label)[cell_ID-1].centroid
    centroid = np.array([x0,y0])
    temp_df = track_df.loc[track_df['particle'] == track_id]
    for index,rows in temp_df.iterrows():
        object_coord = np.array([rows['x'],rows['y']])
        try:
            dst = distance.euclidean(object_coord, centroid)
        except:
            dst = np.inf
        distances.append(dst)
    return min(distances)

# Function Identifies nearby tracks given the track dataframe, a track of interest, and a search radius
def getDistanceFromTrack(track_df, track_id, poi_id):
    poi_df = track_df.loc[track_df['particle'] == poi_id]
    temp_df = track_df.loc[track_df['particle'] == track_id]
    distances = []
    for index,rows in temp_df.iterrows():
        object_coord = np.array([rows['new_x'],rows['new_y']])
        for index_p, rows_p in poi_df.iterrows():
            poi_coord = np.array([rows_p['new_x'], rows_p['new_y']])
            try:
                dst = distance.euclidean(object_coord, poi_coord)

            except:
                dst = np.inf
            
            distances.append(dst)            
    return min(distances)


In [19]:
def getNearbyTrackstoObject(track_df, func, cell_ID=1, min_val = 0, max_val=30, **kwargs):
    nearby_tracks = []
    unique_tracks = np.array(track_df['particle'].unique())
    val = np.infty
    remove=False
    for track_id in unique_tracks:
        if 'axis' in kwargs:
            val = func(track_df, track_id, cell_ID, axis=kwargs['axis'], label_img=kwargs['label'])
        elif 'poi_id' in kwargs:
            val = func(track_df, track_id, poi_id=kwargs['poi_id'])
            remove=True
        else:
            val = func(track_df, track_id, cell_ID, label_img=kwargs['label'])
        if val <= max_val and val >= min_val:
            nearby_tracks.append(track_id)
    if remove == True:
        nearby_tracks.remove(kwargs['poi_id'])
    return nearby_tracks

def createTrackDict(track_df, min_search=10, max_search=200, step_size=30, *track_dict, **kwargs):
    return_dict = {}
    if track_dict:
        return_dict = track_dict
    
    if 'label' in kwargs:
        for i in range(min_search,max_search,step_size):
            print(i, end=" ")
            temp_tracks = getNearbyTrackstoObject(t3, max_val=i, **kwargs)
            return_dict[i] = temp_tracks
    else:    
        unique_tracks = np.array(track_df['particle'].unique())   
        for track_id in unique_tracks:
            return_dict[track_id] = {}
            print(track_id, end=" ")
            for i in range(min_search,max_search,step_size):
                temp_tracks = getNearbyTrackstoObject(t3, poi_id= track_id, max_val=i, **kwargs)
                return_dict[track_id][i] = temp_tracks
    return return_dict

def create_distance_matrix(track_df, unique_tracks):
    return_mat = np.zeros([len(unique_tracks), len(unique_tracks)])
    for i, track_id in enumerate(unique_tracks):
        for j, track_id_2 in enumerate(unique_tracks):
            if i == j:
                continue
            if return_mat[i,j] != 0:
                continue
            else:
                dst = getDistanceFromTrack(track_df, track_id_2, track_id)
                return_mat[i, j] = dst
                return_mat[j, i] = dst
    return return_mat

#dist_mat = create_distance_matrix(temp_cell_df, np.array(temp_cell_df['particle'].unique()))

In [20]:
# Function computes the cosine distance between the track of interest and other tracks specified in an array
def compute_cosine_distance(track_df, poi_id, nearby_tracks):
    # Compute the Cosine Distance to find differences in orientation between all these tracks
    cosine_distances = []
    displacment_vector = []
    
    if isinstance(poi_id, (np.ndarray)):
        displacment_vector = poi_id
    else:
        poi_df = track_df.loc[track_df['particle'] == poi_id]
        #Compute Displacement vector of the particle of interest
        min_particle = poi_df.loc[poi_df['frame'].argmin()]
        max_particle = poi_df.loc[poi_df['frame'].argmax()]
        min_coord = np.array([min_particle['x'], min_particle['y']])
        max_coord = np.array([max_particle['x'], max_particle['y']])
        displacment_vector = np.subtract(max_coord, min_coord)        
    
    for particle_id in nearby_tracks:
        temp_df = track_df.loc[track_df['particle'] == particle_id]
        min_particle = temp_df.loc[temp_df['frame'].argmin()]
        max_particle = temp_df.loc[temp_df['frame'].argmax()]
        min_coord = np.array([min_particle['x'], min_particle['y']])
        max_coord = np.array([max_particle['x'], max_particle['y']])
        disp_vector_2 = np.subtract(max_coord, min_coord)

        cos_dist = np.divide(np.dot(displacment_vector, disp_vector_2), LA.norm(displacment_vector) * LA.norm(disp_vector_2))
        cosine_distances.append([cos_dist, particle_id])
    return cosine_distances

def getOrientationFromAxis(track_df, track_id, cell_ID=1, label_img=None, axis='major'):
    props = regionprops(label_img)[cell_ID-1]
    y0, x0 = props.centroid
    orientation = props.orientation
    
    axis_vec = np.nan
    if axis=='major':
        x1 = x0 + math.cos(orientation) * 0.5 * props.major_axis_length
        y1 = y0 - math.sin(orientation) * 0.5 * props.major_axis_length
        axis_vec = np.array([x1-x0, y1-y0])
    else:
        x2 = x0 - math.sin(orientation) * 0.5 * props.minor_axis_length
        y2 = y0 - math.cos(orientation) * 0.5 * props.minor_axis_length
        axis_vec = np.array([x2-x0, y2-y0])

    cosine_distance = compute_cosine_distance(nearby_tracks=[track_id], track_df=track_df, poi_id=axis_vec)
    return cosine_distance[0][0]

def getOrientationTracksAxis(track_df, min_val=0, max_val=1, cell_ID=1, label_img=None, axis='major'):
    nearby_tracks = []
    unique_tracks = np.array(track_df['particle'].unique())
    for track_id in unique_tracks:
        val = getOrientationFromAxis(track_df, track_id, cell_ID, label_img, axis='major')
        if val <= max_val and val >= min_val:
            nearby_tracks.append(track_id)
    return nearby_tracks


### Load and save data

In [25]:
folders = ['3_23_17_invitro', '11_10_16_invitro', '0836_M17_EB3-appple_4_3_2017_left_ear', 
           '0836_M19_EB3-apple_4_12_2017_no_hole', '0836_M20_EB3-apple_tubulin_MertK-GFP_4_12_2017_right_ear',
          'B8P1', 'C5P1', 'E7P2_1', 'E7P2_2_', 'F8P1_1', 'F8P1_2', 'F10P2', 'invivo_1']

folders = ['collagen_eb3_plate1', 'collagen_eb3_plate2']
folders = ['invivo_2']

In [26]:
#PHASE 1
for folder in folders:
    ##################################################
    # Load all track data
    root_dir = "../../filtered_data_2/" + folder + "/"
    track_dir = root_dir + "track_matrices"
    movie_dir = root_dir + "movie_data"
    mask_dir = root_dir + "masks"
    pickle_dir = root_dir + "pickle_objs/"

    file_names = os.listdir(track_dir)
    
    movies = []
    img_sequences = []
    label_imgs = []
    mask_imgs = []
    movie_names = []
    t0_tracks = []
    
    print("Loading data: " + folder)
    img_id = 0
    for name in file_names:
        new_name = name.split(".csv")[0]
        try:
            #Loading Movie data
            img_sequence = io.imread(movie_dir+"/"+new_name+".tif")
            movies.append(pims.Frame(img_sequence))
            img_sequences.append(img_sequence)
            movie_names.append(name)
            #Loading Mask Data
            img_mask = io.imread(mask_dir+"/"+new_name+".tif")
            mask_imgs.append(img_mask)
            label_imgs.append(label(img_mask))
            #Loading Track Data as dataframes
            t = pd.read_csv(track_dir+"/"+new_name+".csv", header=None)
            t.columns = ["x", "y", "mass", "frame", "particle"]
            t['img_id'] = img_id
            t0_tracks.append(t)
            img_id+=1
        except:
            print(new_name + " was not found")
            
    pickle.dump(file_names, open(pickle_dir + "file_names.p", "wb"))
    
    ##################################################
    print("\tt1_tracks: removing tracks less than a certain length")
    t1_tracks = []
    for i,temp_df in enumerate(t0_tracks):
        num_frames = img_sequences[i].shape[0]
        run_length = 3
        if num_frames >40:
            run_length = 5
        t1 = tp.filter_stubs(temp_df, run_length)
        t1_tracks.append(t1) 
    #print('Before:', temp_df['particle'].nunique(), 'After:',t1['particle'].nunique())

    ##################################################
    print("\tt2_tracks: identify the cell for each track")
    t2_tracks = []
    for i,temp_df in enumerate(t1_tracks):
        #print("Identifying Cells for Track" , i)
        t2 = identifyCellFromTrack(t1_tracks[i], label_imgs[i])
        t2_tracks.append(t2) 

    res_framerate_file = root_dir + "external_data/image_resolution_framerate.csv"
    res_framerate = pd.read_csv(res_framerate_file, header=None)

    t2_2_tracks = []
    for i,temp_df in enumerate(t2_tracks):
        row_val = res_framerate.loc[res_framerate[0] == file_names[i]]
        new_df = pd.concat([temp_df, 
                        pd.DataFrame(columns=['new_x', 'new_y','new_frame', 'delta_t', 'resolution'])])
        res = list(row_val[1])[0]
        framerate = list(row_val[2])[0]
        new_x = np.divide(temp_df['x'], res)
        new_y = np.divide(temp_df['y'], res)
        new_frame = np.multiply(temp_df['frame'], framerate)
        new_df['new_x'] = new_x
        new_df['new_y'] = new_y
        new_df['new_frame'] = new_frame
        new_df['delta_t'] = framerate
        new_df['resolution'] = res
        t2_2_tracks.append(new_df)

    t2_tracks = t2_2_tracks
    pickle.dump(t2_tracks, open(pickle_dir + "t2_tracks.p", "wb"))
    
    ##################################################
    print("\tt2_features: Computing Motion and mass features from each track")   
    t2_features = []
    for i,temp_df in enumerate(t2_tracks):
        #print("Calculating Basic Features for Track", i)
        t2_feat = generate_features_df(temp_df)
        t2_feat = calc_mass_features(temp_df, t2_feat)
        t2_feat = calc_motion_features(temp_df, t2_feat)
        t2_features.append(t2_feat)
        
    pickle.dump(t2_features, open(pickle_dir + "t2_features.p", "wb"))
    
    #t2_tracks = pickle.load(open(pickle_dir + "t2_tracks.p", "rb"))
    #t2_features = pickle.load(open(pickle_dir + "t2_features.p", "rb"))
    
    ##################################################
    print("\tt3_tracks and t3_features: filtering tracks based on features values")
    t3_tracks = []
    t3_features = []
    for i,temp_df in enumerate(t2_tracks):
        #print("Filtering based on Features for Track", i)
        t3 = temp_df
        t3_feat = t2_features[i]
        t3 = filter_track_from_feat(t3, t3_feat, 'new_displacement', min_val=1)
        t3_feat = filter_traj_from_feat(t3_feat, 'new_displacement', min_val=1)

        t3 = filter_track_from_feat(t3, t3_feat, 'new_persistance', min_val=0.60)
        t3_feat = filter_traj_from_feat(t3_feat, 'new_persistance', min_val=0.60)
        #t3 = filter_track_from_feat(t3, t3_feat, 'curvature', max_val = 5)
        #t3_feat = filter_traj_from_feat(t3_feat, 'curvature', max_val = 5)

        t3_tracks.append(t3)
        t3_features.append(t3_feat)

    pickle.dump(t3_tracks, open(pickle_dir + "t3_tracks.p", "wb"))
    pickle.dump(t3_features, open(pickle_dir + "t3_features.p", "wb"))
    
    

Loading data: invivo_2
	t1_tracks: removing tracks less than a certain length
	t2_tracks: identify the cell for each track
	t2_features: Computing Motion and mass features from each track
	t3_tracks and t3_features: filtering tracks based on features values


In [27]:
#PHASE 2
for folder in folders:
    ##################################################
    # Load all track data
    root_dir = "../../filtered_data_2/" + folder + "/"
    track_dir = root_dir + "track_matrices"
    movie_dir = root_dir + "movie_data"
    mask_dir = root_dir + "masks"
    pickle_dir = root_dir + "pickle_objs/"
    file_names = pickle.load(open(pickle_dir + "file_names.p", "rb"))
    
    movies = []
    img_sequences = []
    label_imgs = []
    mask_imgs = []
    movie_names = []
    t0_tracks = []
    
    print("Loading data: " + folder)
    img_id = 0
    for name in file_names:
        new_name = name.split(".csv")[0]
        try:
            #Loading Movie data
            img_sequence = io.imread(movie_dir+"/"+new_name+".tif")
            movies.append(pims.Frame(img_sequence))
            img_sequences.append(img_sequence)
            movie_names.append(name)
            #Loading Mask Data
            img_mask = io.imread(mask_dir+"/"+new_name+".tif")
            mask_imgs.append(img_mask)
            label_imgs.append(label(img_mask))
            #Loading Track Data as dataframes
            t = pd.read_csv(track_dir+"/"+new_name+".csv", header=None)
            t.columns = ["x", "y", "mass", "frame", "particle"]
            t['img_id'] = img_id
            t0_tracks.append(t)
            img_id+=1
        except:
            print(new_name + " was not found")
            
    ##################################################
    print("\tt3_tracks_updated: Calculating Distance Feature")
    t3_tracks = pickle.load(open(pickle_dir + "t3_tracks.p", "rb"))
    t3_features = pickle.load(open(pickle_dir + "t3_features.p", "rb"))
    
    #Computing edge distance
    distance_transforms = []
    for i,temp in enumerate(t3_tracks):
        num_cells = len(np.unique(label_imgs[i])) - 1
        new_dist_transform = []
        for j in range(num_cells):
            cell_id = j+1
            temp_mask = np.multiply(mask_imgs[i],label_imgs[i] == cell_id)
            dist_transf= distance_transform_edt(temp_mask)
            distance_transf_sum = np.sum(dist_transf)
            num_pixels = np.sum(temp_mask)
            if cell_id == 1:
                new_dist_transform = np.divide(dist_transf, (distance_transf_sum/num_pixels))
            else:
                new_dist_transform = np.add(new_dist_transform, np.divide(dist_transf, (distance_transf_sum/num_pixels)))
        distance_transforms.append(new_dist_transform)
    
    t3_temp_features = []
    for i,temp_df in enumerate(t3_tracks):
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['edge_distance'])])
        for index, track_id in enumerate(unique_tracks):
            track_particles = temp_df.loc[temp_df['particle'] == track_id]
            #x and y coordinates are switched in distance trasnform for some reason
            x_vals = np.mean(track_particles['x'])
            y_vals = np.mean(track_particles['y'])
            result = distance_transforms[i][int(y_vals),int(x_vals)]   
            new_df.loc[track_id]['edge_distance'] = result
        t3_temp_features.append(new_df)
    t3_features = t3_temp_features
    
    #Min axis distance and Maj axis distance features
    majaxis_transform = []
    minaxis_transform = []
    for i,temp in enumerate(t3_tracks):
        num_cells = len(np.unique(label_imgs[i])) - 1
        new_maj_transform = np.zeros((mask_imgs[i].shape[0],mask_imgs[i].shape[1]))
        new_min_transform = np.zeros((mask_imgs[i].shape[0],mask_imgs[i].shape[1]))
        for j in range(num_cells):
            cell_id = j+1
            temp_mask = np.multiply(mask_imgs[i],label_imgs[i] == cell_id)

            props = regionprops(temp_mask)
            props = props[0]
            y0, x0 = props.centroid
            orientation = props.orientation
            x1_maj = x0 + math.cos(orientation) * 0.5 * props.major_axis_length
            y1_maj = y0 - math.sin(orientation) * 0.5 * props.major_axis_length
            x2_maj = x0 - math.cos(orientation) * 0.5 * props.major_axis_length
            y2_maj = y0 + math.sin(orientation) * 0.5 * props.major_axis_length

            x1_min = x0 - math.sin(orientation) * 0.5 * props.minor_axis_length
            y1_min = y0 - math.cos(orientation) * 0.5 * props.minor_axis_length
            x2_min = x0 + math.sin(orientation) * 0.5 * props.minor_axis_length
            y2_min = y0 + math.cos(orientation) * 0.5 * props.minor_axis_length

            new_mask_min = np.zeros((temp_mask.shape[0], temp_mask.shape[1]))
            new_mask_maj = np.zeros((temp_mask.shape[0], temp_mask.shape[1]))

            for x_pos in range(temp_mask.shape[0]):
                for y_pos in range(temp_mask.shape[1]):
                    new_mask_min[x_pos,y_pos] = dist(x1_min,y1_min,x2_min,y2_min,y_pos,x_pos)
                    new_mask_maj[x_pos,y_pos] = dist(x1_maj,y1_maj,x2_maj,y2_maj,y_pos,x_pos)

            new_mask_min = np.multiply(temp_mask,new_mask_min)
            max_value = np.max(new_mask_min)
            new_mask_min = np.divide(new_mask_min, max_value)

            new_mask_maj = np.multiply(temp_mask,new_mask_maj)
            max_value = np.max(new_mask_maj)
            new_mask_maj = np.divide(new_mask_maj, max_value)

            new_maj_transform = np.add(new_mask_maj, new_maj_transform)
            new_min_transform = np.add(new_mask_min, new_min_transform)

        majaxis_transform.append(new_maj_transform)
        minaxis_transform.append(new_min_transform)
        t3_temp_features = []
        
    for i,temp_df in enumerate(t3_tracks):
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['maj_axis_distance', 'min_axis_distance'])])
        for index, track_id in enumerate(unique_tracks):
            track_particles = temp_df.loc[temp_df['particle'] == track_id]
            #x and y coordinates are switched in distance trasnform for some reason
            x_vals = np.mean(track_particles['x'])
            y_vals = np.mean(track_particles['y'])
            result_major = majaxis_transform[i][int(y_vals),int(x_vals)]   
            result_minor = minaxis_transform[i][int(y_vals),int(x_vals)]   
            new_df.loc[track_id]['maj_axis_distance'] = result_major
            new_df.loc[track_id]['min_axis_distance'] = result_minor
        t3_temp_features.append(new_df)
    t3_features = t3_temp_features
    
    pickle.dump(t3_tracks, open(pickle_dir + "t3_tracks.p", "wb"))
    pickle.dump(t3_features, open(pickle_dir + "t3_features.p", "wb"))
    
    ##################################################
    #t3_tracks = pickle.load(open(pickle_dir + "t3_tracks.p", "rb"))
    #t3_features = pickle.load(open(pickle_dir + "t3_features.p", "rb"))
    
    print("\tcreating track dictionaries")
    track_distance_dicts = {}

    for i,temp_df in enumerate(t3_tracks):
        track_distance_dicts[i] = {}
        unique_cells =  np.array(temp_df['cell_id'].unique())

        for cell in unique_cells:
            #track_distance_dicts[i][cell] = {}
            #print("Creating Dictionary for Cell", cell, "in Image", i)
            temp_cell_df = temp_df.loc[temp_df['cell_id'] == cell]
            unique_tracks = np.array(temp_cell_df['particle'].unique())
            dist_mat = create_distance_matrix(temp_cell_df, unique_tracks)

            for index,track_id in enumerate(unique_tracks):
                #track_distance_dicts[i][cell][track_id] = {}
                track_distance_dicts[i][track_id] = {}

                for srange in [5,10,20,100,200]:
                    nearby_tracks = [unique_tracks[j] for j, val in enumerate(dist_mat[index,:]) 
                                     if (val<srange and j != index)]
                    #track_distance_dicts[i][cell][track_id][srange] = nearby_tracks
                    track_distance_dicts[i][track_id][srange] = nearby_tracks  
                    
    pickle.dump(track_distance_dicts, open(pickle_dir + "track_distance_dicts.p", "wb"))
    #track_distance_dicts = pickle.load(open(pickle_dir + "track_distance_dicts.p", "rb"))
    
    ##################################################
    # Get Features from Plots Above that Represent Orientation
    print("\tt4_features - Get similarity features")

    t4_features = []
    for i,temp_df in enumerate(t3_tracks):
        #print("Computing Similarity Features for Image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t3_features[i],pd.DataFrame(columns=['similarity_5','similarity_10','similarity_20', 'similarity_100', 'similarity_200'])])
        for index, track_id in enumerate(unique_tracks):
            nearby_tracks = track_distance_dicts[i][track_id][5]
            if nearby_tracks == []:
                continue
            else:
                distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
                average_distance = np.average(np.array(distances)[:,0])
                new_df.loc[track_id]['similarity_5'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][10]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.loc[track_id]['similarity_10'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][20]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.loc[track_id]['similarity_20'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][100]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.loc[track_id]['similarity_100'] = average_distance

            nearby_tracks = track_distance_dicts[i][track_id][200]
            distances = compute_cosine_distance(temp_df, track_id, nearby_tracks)
            average_distance = np.average(np.array(distances)[:,0])
            new_df.loc[track_id]['similarity_200'] = average_distance
        t4_features.append(new_df)
        
    pickle.dump(t4_features, open(pickle_dir + "t4_features.p", "wb"))
    #t4_features = pickle.load(open(pickle_dir + "t4_features.p", "rb"))

    ##################################################
    print("\tt5_features - Get Orientation Features")
    t5_features = []
    for i,temp_df in enumerate(t3_tracks):
        #print("Computing Major/Minor Axis Features for Image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t4_features[i],pd.DataFrame(columns=['maj_orient', 'min_orient', 'maj_orient_mag', 'min_orient_mag'])])
        for index, track_id in enumerate(unique_tracks):
            cell_ID = int(temp_df.loc[temp_df['particle']==track_id]['cell_id'].unique()[0])
            distance_maj = getOrientationFromAxis(temp_df, track_id, cell_ID=cell_ID, label_img=label_imgs[i], axis='major')
            distance_min = getOrientationFromAxis(temp_df, track_id, cell_ID=cell_ID, label_img=label_imgs[i], axis='minor')
            new_df.loc[track_id]['maj_orient'] = distance_maj
            new_df.loc[track_id]['min_orient'] = distance_min
            new_df.loc[track_id]['maj_orient_mag'] = np.absolute(distance_maj)
            new_df.loc[track_id]['min_orient_mag'] = np.absolute(distance_min)

        t5_features.append(new_df)
        
    t6_features = []
    ##################################################
    print("\tt6_features - Creating Cell IDs")
    for i,temp_df in enumerate(t3_tracks):
        #print("Creating Unique Cell ID for image",  i)
        unique_tracks = np.array(temp_df['particle'].unique())
        new_df = pd.concat([t5_features[i],pd.DataFrame(columns=['unique_cell_id', 'unique_id', 'img_id'])])
        for index, track_id in enumerate(unique_tracks):
            cell_ID = int(temp_df.loc[temp_df['particle']==track_id]['cell_id'].unique()[0])
            new_cell_id = float(str(i)+ '.'+str(cell_ID))
            new_df.loc[track_id]['unique_cell_id'] = new_cell_id
            new_part_id = float(str(i)+'.'+str(int(track_id)))
            new_df.loc[track_id]['unique_id'] = new_part_id
            new_df.loc[track_id]['img_id'] = i
        t6_features.append(new_df.dropna())
        
    pickle.dump(t6_features, open(pickle_dir + "t6_features.p", "wb"))
    #t6_features = pickle.load(open(pickle_dir + "t6_features.p", "rb"))

Loading data: invivo_2
	t3_tracks_updated: Calculating Distance Feature
	creating track dictionaries
	t4_features - Get similarity features
	t5_features - Get Orientation Features
	t6_features - Creating Cell IDs


# END